In [20]:
import os

In [7]:
# Let latest version from github
os.system("git remote update");
os.system("git pull");

In [21]:
# Adds/commits this notebook
def update(message):
    os.system("git add ML_projekt1.ipynb")
    os.system('git commit -m "{}"'.format(message))
    os.system("git push")

In [133]:
update("Outline of report written")

# Project starts here

In [111]:
import numpy as np
import pandas as pd
#df = pd.read_csv("tmdb_5000_movies.csv")
df = pd.read_csv("movie_metadata.csv")
df = df.dropna();
df.head(1)

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000


In [101]:
int(df.shape[0]) - int(df.dropna().shape[0])

1287

# Dataset introduction and description

The IMDB Movie Dataset from Kaggle.com, contains information from 5000 movies, obtained from webscraping IMDB.com.

In this dataset we will explore what features determine a "successful" movie. The definition of "successful" is of course a bit vague, but during this dataanalysis we will explore... different success criterias such as popularity and revenue.

In our initial dataanalysis we will use regression to detemine if there is a correlation between IMDB score and revenue.

1. A description of your data set.
Explain
• What the problem of interest is (i.e. what is your data about),
• Where you obtained the data,
• What has previously been done to the data. (i.e. if available go through
some of the original source papers and read what they did to the data
and summarize what were their results).
• What the primary machine learning modeling aim is for the data, i.e.
which attributes you feel are relevant when carrying out a classification,
a regression, a clustering, an association mining, and an anomaly detection
in the later reports and what you hope to accomplish using these
techniques. For instance, which attribute do you wish to explain in the
regression based on which other attributes? Which class label will you predict based on which other attributes in the classification task? If you need to transform the data to admit these tasks, explain roughly how you might do this (but don’t transform the data now!).


To get an overview of the dataset columns, we can look at the first entry in our dataframe.

In [81]:
# Check datatypes
df.dtypes

color                         object
director_name                 object
num_critic_for_reviews       float64
duration                     float64
director_facebook_likes      float64
actor_3_facebook_likes       float64
actor_2_name                  object
actor_1_facebook_likes       float64
gross                        float64
genres                        object
actor_1_name                  object
movie_title                   object
num_voted_users                int64
cast_total_facebook_likes      int64
actor_3_name                  object
facenumber_in_poster         float64
plot_keywords                 object
movie_imdb_link               object
num_user_for_reviews         float64
language                      object
country                       object
content_rating                object
budget                       float64
title_year                   float64
actor_2_facebook_likes       float64
imdb_score                   float64
aspect_ratio                 float64
m

2. A detailed explanation of the attributes of the data.
• Describe if the attributes are discrete/continous, Nominal/Ordinal/Interval/Ratio,
• give an account of whether there are data issues (i.e. missing values or
corrupted data) and describe them if so
• describe the basic summary statistics of the attributes.
If your data set contains many similar attributes, you may restrict yourself to
describing a few representative features (apply common sense).


3. Data visualization(s) based on suitable visualization techniques including
a principal component analysis (PCA).
Touch upon the following subjects, use visualizations when it appears sensible.
Keep in mind the ACCENT principles and Tufte’s guidelines when you visualize
the data.
• Are there issues with outliers in the data,
• do the attributes appear to be normal distributed,
• are variables correlated,
• does the primary machine learning modeling aim appear to be feasible
based on your visualizations.
There are three aspects that needs to be described when you carry out the
PCA analysis for the report:
• The amount of variation explained as a function of the number of PCA
components included,
• the principal directions of the considered PCA components (either find a
way to plot them or interpret them in terms of the features),
• the data projected onto the considered principal components.
If your attributes have very different scales it may be relevant to standardize
the data prior to the PCA analysis.

## Principle Component Analysis

In this chapter we will explore the singular values of the dataset, through Principle Component Analysis.

In [56]:
from sklearn.decomposition import PCA

Since our dataset contains categorical data, we need to convert these columns to one-hot notation. To do this we create a function which converts all columns containing object data, to multiple columns with 1 or 0.

In [123]:
def toonehot(df):
    obj_df = df.select_dtypes(include=['object']).copy()
    obj_df[obj_df.isnull().any(axis=1)]
    cols = obj_df.columns.values.tolist()
    df2 = pd.get_dummies(obj_df, columns=cols, prefix=cols)
    df4 = pd.concat([df2, df.drop(cols, 1)], 1)
    
    return df4

In [126]:
ohdf = toonehot(df)
ohdf.head()

,color_ Black and White,color_Color,director_name_Aaron Schneider,director_name_Aaron Seltzer,director_name_Abel Ferrara,director_name_Adam Goldberg,director_name_Adam Marcus,director_name_Adam McKay,director_name_Adam Rapp,director_name_Adam Rifkin,...,num_voted_users,cast_total_facebook_likes,facenumber_in_poster,num_user_for_reviews,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,0,1,0,0,0,0,0,0,0,0,...,886204,4834,0.0,3054.0,237000000.0,2009.0,936.0,7.9,1.78,33000
1,0,1,0,0,0,0,0,0,0,0,...,471220,48350,0.0,1238.0,300000000.0,2007.0,5000.0,7.1,2.35,0
2,0,1,0,0,0,0,0,0,0,0,...,275868,11700,1.0,994.0,245000000.0,2015.0,393.0,6.8,2.35,85000
3,0,1,0,0,0,0,0,0,0,0,...,1144337,106759,0.0,2701.0,250000000.0,2012.0,23000.0,8.5,2.35,164000
5,0,1,0,0,0,0,0,0,0,0,...,212204,1873,1.0,738.0,263700000.0,2012.0,632.0,6.6,2.35,24000


From the dataframe head, we can see that the columns are now split out over multiple ... columns .... We can check this by selecting columns with the datatype 'object'.

In [131]:
ohdf.select_dtypes(include=['object']).head(1)

""
0


Now that our dataset is cleaned up and converted to the correct datatypes, we can run the PCA using scikit learn.

In [132]:
pca = PCA(n_components=5, svd_solver = 'full')
pca.fit(ohdf)
pca.components_ 

array([[ -1.22948006e-11,   1.22947025e-11,  -2.09364519e-13, ...,
          1.46863933e-10,   4.04649219e-11,   5.34702099e-06],
       [ -1.06568159e-10,   1.06567900e-10,  -2.29177538e-12, ...,
          3.21066909e-09,   3.10334156e-10,   1.11389880e-04],
       [  1.18625373e-07,  -1.18625372e-07,  -5.22977136e-10, ...,
          3.96284771e-06,   1.72605716e-07,   6.82985836e-02],
       [ -2.62498412e-07,   2.62498393e-07,   6.04357497e-09, ...,
          1.94678470e-07,   7.87422320e-07,   1.26862279e-01],
       [ -3.94738954e-07,   3.94738955e-07,  -3.35070412e-09, ...,
          2.23190955e-06,   1.31958973e-06,   9.88765134e-01]])

In [122]:
normalized_singular_values = list(pca.singular_values_/sum(pca.singular_values_))
normalized_singular_values

[0.76337773629957828,
 0.23607880909400641,
 0.00040173915128316896,
 8.0326568196162961e-05,
 6.1388886936007673e-05]

Principle Component Analysis reveals that the two first components account for close to 90% og the total variance.

In [73]:
df[['gross']['director_name' == 'James Cameron']].head(8)

0    760505847.0
1    309404152.0
2    200074175.0
3    448130642.0
4            NaN
5     73058679.0
6    336530303.0
7    200807262.0
Name: gross, dtype: float64

## Descriptive Analysis

Analysis of relation between facebook likes and earnings

In [ ]:
work = df[['movie_facebook_likes','gross']].dropna();
work = work[(work != 0).all(1)];
work.plot.scatter('movie_facebook_likes', 'gross', grid=True);

Relation between movie_facebook_likes imdb_score

In [ ]:
work.nlargest(5, 'movie_facebook_likes')

In [ ]:
work = df[['imdb_score', 'movie_facebook_likes']].dropna();
work = work[(work != 0).all(1)];
work.plot.scatter('imdb_score', 'movie_facebook_likes', grid=True);

In [ ]:
dfc = df.corr()
np.fill_diagonal(dfc.values, 0)
dfc

In [ ]:
dfc.min()

In [ ]:
work = df[['imdb_score', 'movie_facebook_likes']].dropna();
work = work[(work != 0).all(1)];
work.plot.scatter('imdb_score', 'movie_facebook_likes', grid=True);